In [ ]:
import SAGA
from easyquery import Query, QueryMaker
from SAGA.database import FitsTable
from SAGA import ObjectCuts as C
from SAGA import HostCuts as H
from astropy.table import join
from SAGA.utils.distance import z2v
from SAGA.utils import join_str_arr
from SAGA.objects.cuts import _known_telnames as pre_exist_tel

In [ ]:
saga = SAGA.QuickStart()
saga.set_default_base_version("paper2")

In [ ]:
base = SAGA.database.FitsTable("/home/yymao/Downloads/saga_base_all.fits").read()
base = saga.host_catalog.construct_host_query("paper2_complete").filter(base)
assert len(np.unique(base["HOSTID"])) == 36

sats = Query(C.is_sat).filter(base)
del base

In [ ]:
assert len(sats) == 127

In [ ]:
sats.sort(["HOST_RA", "r_mag"])

In [ ]:
def format_tel(s):
    s = s.split("+")
    if len(s) == 1:
        return s[0]
    exist = sorted(pre_exist_tel.intersection(s))
    if not exist:
        return sorted(s)[0] + "+"
    
    while exist:
        if len(exist) == 1:
            return exist[0]
        if 'NSA' in exist:
            exist.remove("NSA")
            continue
        if 'GAMA' in exist:
            exist.remove("GAMA")
            continue
        if "SDSS" in exist:
            return "SDSS+"
        return sorted(exist)[0] + "+"

In [ ]:
t = sats['HOST_COMMON_NAME', "OBJID", "survey", 'RA', "DEC", 'RHOST_KPC', 'SPEC_Z', "HOST_VHOST", 'r_mag', 'Mr', 'gr', 'sb_r', 'log_sm', "SPEC_REPEAT"]


t["OBJID"] = np.where(
    t["survey"] == "sdss", 
    t["OBJID"], 
    np.where(
        t["survey"] == "decals", 
        join_str_arr("LS-", (sats["OBJID"] // int(1e13)).astype(str), "-", (sats["OBJID"] % int(1e13)).astype(str)),
        join_str_arr(np.char.upper(sats["survey"]), "-", sats["OBJID"].astype(str)),
    ),
)
del t["survey"]
t["SPEC_REPEAT"] = list(map(format_tel, t["SPEC_REPEAT"]))
t['RA'].format = '%.4f'
t['DEC'].format = '%.4f'
t['RHOST_KPC'].format = '%d'
t["SPEC_Z"] = z2v(t["SPEC_Z"]) - t["HOST_VHOST"]
t["SPEC_Z"].format = '%.d'
del t["HOST_VHOST"]
t['r_mag'].format = '%.2f'
t['Mr'].format = '%.1f'
t['gr'].format = '%.2f'
t['sb_r'].format = '%.2f'
t['log_sm'].format = '%.2f'


t.rename_column("RHOST_KPC", "D_PROJ")
t.rename_column("SPEC_Z", "DELTA_HRV")
t.rename_column("r_mag", "R")
t.rename_column("Mr", "R_ABS")
t.rename_column("gr", "GR")
t.rename_column("sb_r", "MU_EFF")
t.rename_column("log_sm", "LOG_STELLAR_MASS")
t.rename_column("SPEC_REPEAT", "Z_SOURCE")

In [ ]:
assert (
    np.count_nonzero(np.char.startswith(t["Z_SOURCE"], "MMT")) + 
    np.count_nonzero(np.char.startswith(t["Z_SOURCE"], "AAT")) + 
    np.count_nonzero(np.char.startswith(t["Z_SOURCE"], "PAL"))) == 69

In [ ]:
colheads = [
    "Host\nName",
    "Object\nID",
    "R.A.\n[deg]",
    "Dec.\n[deg]",
    "$d_\\text{proj}$\n[kpc]",
    "$\\Delta$\\,HRV\n[\\kms]",
    "$r_o$\n[mag]",
    "$M_{r,o}$\n[mag]",
    "$(g-r)_o$\n[mag]",
    "$\\mueff$\n[mag\,$('')^{-2}$]", 
    "$M_*$\n[\\msun]",
    "Redshift\nsource",
]

In [ ]:
colheads_formatted = [[], [], []]
for i, colhead in enumerate(colheads):
    colhead = colhead.splitlines()
    while len(colhead) < 2:
        colhead.insert(0, "~")
    colhead.append(f"({i+1})")
    for new, l in zip(colheads_formatted, colhead):
        new.append(l)

In [ ]:
print("\\\\[-6pt]\n".join((" & ".join((f"\colhead{{{i}}}" for i in row)) for row in colheads_formatted)))

In [ ]:
from io import StringIO
o = StringIO()
t.write(o, format="ascii.aastex")
o.seek(0)

host = None
cached_line = None
for line in o:
    line = line.strip()
    if line[0] == '\\':
        if cached_line:
            print(cached_line)
            cached_line = None
        continue
    if host != line.partition(" ")[0]:
        if host is not None:
            cached_line = cached_line + " \\hline"
        host = line.partition(" ")[0]
    if cached_line:
        print(cached_line)
        cached_line = None
    cached_line = line

In [ ]:
t["INTERNAL_HOSTID"] = sats["HOSTID"]
t["INTERNAL_OBJID"] = sats["OBJID"]

In [ ]:
t.write("/home/yymao/Downloads/saga_stage2_sats.csv", overwrite=True)